# BAI Indexed Region Queries (v1.6.0)

**Tutorial Duration**: 30-40 minutes

**What You'll Learn**:
- How to load and use BAI indexes for fast region queries
- Performance comparison: indexed vs full scan queries
- Common workflows: coverage analysis, multi-region queries
- When to use indexed queries vs sequential reading

**Prerequisites**:
- Basic Python knowledge
- Familiarity with BAM files (see Tutorial 05 if needed)
- biometal v1.6.0+ installed: `pip install biometal-rs`

## What is BAI Indexing?

**BAI (BAM Index)** is a format that enables **O(log n) random access** to specific genomic regions in BAM files, compared to O(n) for full scans.

### Key Concepts

1. **Hierarchical Binning**: Divides genome into 6 levels of bins (37,450 total)
2. **Linear Index**: 16 Kbp intervals provide minimum offsets
3. **Virtual Offsets**: Combine compressed file position + decompressed block offset

### Performance Impact

| File Size | Speedup |
|-----------|--------|
| 100 MB | 1.68× |
| 1 GB | 10-20× |
| 10 GB | 100-200× |
| 100 GB | 500×+ |

**Index Loading**: <1ms (negligible overhead)

## Setup

First, let's import biometal and verify we have v1.6.0+:

In [ ]:
import biometal
import time
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import numpy as np

print(f"biometal version: {biometal.__version__}")
assert biometal.__version__ >= "1.6.0", "Please upgrade: pip install --upgrade biometal-rs"

## Test Data

For this tutorial, we'll use the synthetic test BAM file included with biometal. In production, you'd use your own BAM files with corresponding `.bai` indexes.

In [ ]:
# Paths to test data (adjust for your system)
BAM_PATH = "../tests/data/synthetic_100k.bam"
BAI_PATH = "../tests/data/synthetic_100k.bam.bai"

# Verify files exist
import os
print(f"BAM file exists: {os.path.exists(BAM_PATH)}")
print(f"BAI file exists: {os.path.exists(BAI_PATH)}")

## Section 1: Loading and Inspecting BAI Index

### 1.1 Load BAI Index

In [ ]:
# Load BAI index (typically <1ms)
start_time = time.time()
index = biometal.BaiIndex.from_path(BAI_PATH)
load_time = (time.time() - start_time) * 1000  # Convert to milliseconds

print(f"Index loaded in {load_time:.2f} ms")
print(f"Number of references: {index.reference_count}")
print(f"Index representation: {repr(index)}")

**Key Takeaway**: Index loading is extremely fast (<1ms), so you can load it every time without worrying about overhead.

## Section 2: Basic Indexed Queries

### 2.1 Simple Region Query

In [ ]:
# Query a small region on chr1
query_start = 1
query_end = 1000

print(f"Querying chr1:{query_start}-{query_end}...")

count = 0
high_quality = 0

for record in biometal.BamReader.query_region(
    BAM_PATH,
    index,
    "chr1",
    query_start,
    query_end
):
    count += 1
    if record.mapq and record.mapq >= 30:
        high_quality += 1

print(f"Total reads: {count:,}")
print(f"High quality (Q≥30): {high_quality:,} ({high_quality/count*100:.1f}%)")

### 2.2 Understanding Overlapping Reads

**Important**: Indexed queries return reads that **overlap** the region, not just reads that **start** in the region.

In [ ]:
# Example: Query a single base position
single_base_start = 100
single_base_end = 101  # BAM uses half-open intervals

overlapping_reads = 0
for record in biometal.BamReader.query_region(
    BAM_PATH,
    index,
    "chr1",
    single_base_start,
    single_base_end
):
    overlapping_reads += 1

print(f"Reads overlapping position {single_base_start}: {overlapping_reads:,}")
print("\nThis includes:")
print("  - Reads starting before position 100 but extending past it")
print("  - Reads starting exactly at position 100")
print("  - This is standard BAM behavior (matches samtools)")

## Section 3: Performance Comparison

### 3.1 Indexed Query vs Full Scan

In [ ]:
# Define region to query
region_ref = "chr1"
region_start = 1000
region_end = 2000

# METHOD 1: Indexed query (O(log n))
start = time.time()
indexed_count = 0
for record in biometal.BamReader.query_region(
    BAM_PATH,
    index,
    region_ref,
    region_start,
    region_end
):
    indexed_count += 1
indexed_time = time.time() - start

# METHOD 2: Full scan with manual filter (O(n))
start = time.time()
scan_count = 0
for record in biometal.BamReader.from_path(BAM_PATH):
    # Manual overlap check
    if (record.reference_id == 0 and  # chr1
        record.position is not None):
        # Calculate read end position
        read_end = record.position
        for op in record.cigar:
            if op.consumes_reference():
                read_end += op.length
        
        # Check if read overlaps region
        if record.position < region_end and read_end > region_start:
            scan_count += 1
scan_time = time.time() - start

# Results
print(f"=== Performance Comparison ===")
print(f"\nIndexed query:")
print(f"  Time: {indexed_time*1000:.1f} ms")
print(f"  Reads: {indexed_count:,}")
print(f"\nFull scan:")
print(f"  Time: {scan_time*1000:.1f} ms")
print(f"  Reads: {scan_count:,}")
print(f"\nSpeedup: {scan_time/indexed_time:.2f}×")
print(f"\nNote: Speedup increases dramatically with file size!")

## Section 4: Multi-Region Queries

### 4.1 Querying Multiple Regions Efficiently

**Key Pattern**: Load index once, reuse for multiple queries

In [ ]:
# Define multiple regions (e.g., exons)
regions = [
    ("chr1", 1000, 2000),
    ("chr1", 5000, 6000),
    ("chr1", 10000, 11000),
    ("chr1", 15000, 16000),
    ("chr1", 20000, 21000),
]

print(f"Querying {len(regions)} regions...\n")

results = []
for ref_name, start, end in regions:
    count = 0
    total_bases = 0
    
    for record in biometal.BamReader.query_region(
        BAM_PATH,
        index,
        ref_name,
        start,
        end
    ):
        count += 1
        total_bases += len(record.sequence)
    
    results.append((ref_name, start, end, count, total_bases))
    print(f"{ref_name}:{start:>6,}-{end:>6,}: {count:>5,} reads, {total_bases:>8,} bases")

# Summary
total_reads = sum(r[3] for r in results)
total_bases = sum(r[4] for r in results)
print(f"\nTotal across all regions: {total_reads:,} reads, {total_bases:,} bases")

## Section 5: Coverage Analysis

### 5.1 Calculate Coverage for Specific Region

In [ ]:
# Query region for coverage analysis
cov_start = 1000
cov_end = 2000

print(f"Calculating coverage for chr1:{cov_start}-{cov_end}...")

# Track coverage at each position
coverage = defaultdict(int)

for record in biometal.BamReader.query_region(
    BAM_PATH,
    index,
    "chr1",
    cov_start,
    cov_end
):
    if not record.is_mapped or record.position is None:
        continue
    
    # Count coverage for each base
    pos = record.position
    for op in record.cigar:
        if op.consumes_reference():
            for i in range(op.length):
                if cov_start <= pos < cov_end:
                    coverage[pos] += 1
                pos += 1
        elif op.consumes_query():
            pass  # Insertions don't advance reference position

# Calculate statistics
if coverage:
    coverage_values = list(coverage.values())
    mean_cov = np.mean(coverage_values)
    median_cov = np.median(coverage_values)
    min_cov = min(coverage_values)
    max_cov = max(coverage_values)
    
    print(f"\nCoverage Statistics:")
    print(f"  Mean: {mean_cov:.2f}×")
    print(f"  Median: {median_cov:.2f}×")
    print(f"  Min: {min_cov}×")
    print(f"  Max: {max_cov}×")
    print(f"  Positions covered: {len(coverage):,}/{cov_end-cov_start:,}")
else:
    print("No coverage data found")

### 5.2 Visualize Coverage

In [ ]:
if coverage:
    # Prepare data for plotting
    positions = sorted(coverage.keys())
    depths = [coverage[pos] for pos in positions]
    
    # Plot
    plt.figure(figsize=(12, 4))
    plt.plot(positions, depths, linewidth=0.5)
    plt.fill_between(positions, depths, alpha=0.3)
    plt.xlabel("Position on chr1")
    plt.ylabel("Coverage Depth")
    plt.title(f"Coverage: chr1:{cov_start}-{cov_end}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Coverage distribution
    plt.figure(figsize=(8, 4))
    plt.hist(depths, bins=50, edgecolor='black')
    plt.axvline(mean_cov, color='red', linestyle='--', label=f'Mean: {mean_cov:.1f}×')
    plt.xlabel("Coverage Depth")
    plt.ylabel("Number of Positions")
    plt.title("Coverage Distribution")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Section 6: Real-World Workflow Examples

### 6.1 Extract High-Quality Reads from Specific Regions

In [ ]:
# Target regions (e.g., known disease-associated loci)
target_regions = [
    ("chr1", 10000, 11000),
    ("chr1", 15000, 16000),
]

quality_threshold = 30

print(f"Extracting high-quality reads (MAPQ ≥ {quality_threshold}) from target regions...\n")

high_quality_reads = []

for ref_name, start, end in target_regions:
    region_hq_count = 0
    
    for record in biometal.BamReader.query_region(
        BAM_PATH,
        index,
        ref_name,
        start,
        end
    ):
        if record.is_mapped and record.mapq and record.mapq >= quality_threshold:
            high_quality_reads.append({
                'name': record.name,
                'region': f"{ref_name}:{start}-{end}",
                'position': record.position,
                'mapq': record.mapq,
                'length': len(record.sequence)
            })
            region_hq_count += 1
    
    print(f"{ref_name}:{start}-{end}: {region_hq_count:,} high-quality reads")

print(f"\nTotal high-quality reads extracted: {len(high_quality_reads):,}")
print(f"\nFirst 5 reads:")
for read in high_quality_reads[:5]:
    print(f"  {read['name']}: {read['region']} @ {read['position']} (Q{read['mapq']})")

### 6.2 Variant Detection Support Data

In [ ]:
# Query region around potential variant
variant_pos = 1500
window = 50  # 50bp window around variant

var_start = variant_pos - window
var_end = variant_pos + window

print(f"Analyzing region around position {variant_pos} (window: ±{window}bp)\n")

# Collect base calls at variant position
base_calls = Counter()
qualities = []
strand_counts = {'forward': 0, 'reverse': 0}

for record in biometal.BamReader.query_region(
    BAM_PATH,
    index,
    "chr1",
    var_start,
    var_end
):
    if not record.is_mapped or record.position is None:
        continue
    
    # Find if this read covers the variant position
    read_pos = record.position
    seq_idx = 0
    
    for op in record.cigar:
        if op.op_char == 'M' or op.op_char == '=':
            for i in range(op.length):
                if read_pos == variant_pos and seq_idx < len(record.sequence):
                    base = chr(record.sequence[seq_idx])
                    qual = record.quality[seq_idx]
                    base_calls[base] += 1
                    qualities.append(qual)
                    
                    # Track strand
                    if record.is_reverse:
                        strand_counts['reverse'] += 1
                    else:
                        strand_counts['forward'] += 1
                    break
                read_pos += 1
                seq_idx += 1
        elif op.consumes_reference():
            read_pos += op.length
        elif op.consumes_query():
            seq_idx += op.length

# Report
if base_calls:
    total = sum(base_calls.values())
    print(f"Base calls at position {variant_pos}:")
    for base, count in base_calls.most_common():
        pct = count / total * 100
        print(f"  {base}: {count:>4} ({pct:>5.1f}%)")
    
    print(f"\nStrand bias:")
    print(f"  Forward: {strand_counts['forward']} ({strand_counts['forward']/total*100:.1f}%)")
    print(f"  Reverse: {strand_counts['reverse']} ({strand_counts['reverse']/total*100:.1f}%)")
    
    if qualities:
        print(f"\nQuality scores:")
        print(f"  Mean: Q{np.mean(qualities):.1f}")
        print(f"  Min: Q{min(qualities)}")
        print(f"  Max: Q{max(qualities)}")
else:
    print("No reads cover this position")

## Section 7: Best Practices

### 7.1 When to Use Indexed Queries

**✅ Use indexed queries when**:
- Extracting specific regions (exons, genes, intervals)
- File size > 100 MB
- Analyzing targeted sequencing data
- Multi-sample region comparison
- Coverage analysis for specific loci
- Variant calling on known positions

**❌ Use sequential reading when**:
- Processing entire BAM file
- File size < 100 MB  
- Genome-wide statistics
- No repeated access to same regions
- Computing global metrics (total reads, overall coverage)

### 7.2 Performance Tips

In [ ]:
# TIP 1: Load index once, reuse for multiple queries
index = biometal.BaiIndex.from_path(BAI_PATH)  # Load once

for region in regions:
    for record in biometal.BamReader.query_region(
        BAM_PATH, index, *region  # Reuse index
    ):
        pass  # Process

# TIP 2: For very large region lists, batch by chromosome
regions_by_chr = defaultdict(list)
for ref, start, end in all_regions:
    regions_by_chr[ref].append((start, end))

# Process chromosome at a time
for ref in sorted(regions_by_chr.keys()):
    for start, end in regions_by_chr[ref]:
        # Query...
        pass

# TIP 3: Use streaming, don't accumulate
# ❌ BAD: records = list(query)  # Loads all into memory
# ✅ GOOD: for record in query: process(record)  # Constant memory

print("Tips applied successfully!")

### 7.3 Error Handling

In [ ]:
# Example: Robust query with error handling

try:
    # Load index
    index = biometal.BaiIndex.from_path(BAI_PATH)
except FileNotFoundError:
    print("Error: BAI index not found. Generate with: samtools index file.bam")
except Exception as e:
    print(f"Error loading index: {e}")
else:
    try:
        # Query region
        count = 0
        for record in biometal.BamReader.query_region(
            BAM_PATH,
            index,
            "chr1",
            1000,
            2000
        ):
            count += 1
        
        print(f"Successfully queried region: {count} reads")
    
    except ValueError as e:
        print(f"Invalid parameters: {e}")
    except Exception as e:
        print(f"Error during query: {e}")

## Section 8: Performance Benchmarking

### 8.1 Measure Your Own Performance

In [ ]:
# Benchmark indexed query performance
import time

regions_to_test = [
    ("chr1", 1, 1000),
    ("chr1", 1, 10000),
    ("chr1", 1, 100000),
]

print("Performance Benchmarking\n" + "="*50)

for ref, start, end in regions_to_test:
    region_size = end - start
    
    # Time indexed query
    start_time = time.time()
    count = sum(1 for _ in biometal.BamReader.query_region(
        BAM_PATH, index, ref, start, end
    ))
    elapsed = time.time() - start_time
    
    print(f"\nRegion: {ref}:{start:>6,}-{end:>6,} ({region_size:>6,} bp)")
    print(f"  Time: {elapsed*1000:.1f} ms")
    print(f"  Reads: {count:,}")
    print(f"  Throughput: {count/elapsed:.0f} reads/sec")

## Summary

In this tutorial, you learned:

1. **BAI Basics**: How to load and use BAI indexes for O(log n) queries
2. **Performance**: 1.68-500× speedup compared to full scan (scales with file size)
3. **Multi-Region Queries**: Efficient pattern for querying multiple regions
4. **Coverage Analysis**: Calculate and visualize coverage for specific regions
5. **Real-World Workflows**: Variant detection, quality filtering, targeted extraction
6. **Best Practices**: When to use indexed queries, performance tips, error handling

### Key Takeaways

- Index loading is negligible (<1ms)
- Load once, reuse for multiple queries
- Speedup increases dramatically with file size
- Use for targeted analysis, not genome-wide statistics
- Streaming architecture maintains constant ~5 MB memory

### Next Steps

- Try with your own BAM files (generate index: `samtools index file.bam`)
- Explore multi-sample comparisons
- Integrate with variant calling pipelines
- Combine with other biometal operations (quality filtering, CIGAR analysis)

### Resources

- **User Guide**: [docs/USER_GUIDE.md](../docs/USER_GUIDE.md)
- **API Docs**: [docs.rs/biometal](https://docs.rs/biometal)
- **BAI Performance**: [docs/BAI_PERFORMANCE_FINDINGS.md](../BAI_PERFORMANCE_FINDINGS.md)
- **Python Tests**: [tests/python/test_bai_index.py](../tests/python/test_bai_index.py)

---

**Tutorial Complete!** 🎉

biometal v1.6.0 - ARM-native bioinformatics  
https://github.com/shandley/biometal